In [1]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw
import math
import os
import sys

#import matplotlib.pyplot as plt

from skimage.color import rgb2gray
from skimage.io import imread, imsave
from skimage.filters import threshold_otsu
from skimage import img_as_uint
from skimage.transform import resize
from skimage.util import invert
from skimage.morphology import skeletonize
from skimage import data
from skimage import morphology

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import model_from_json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle

Using TensorFlow backend.


<h1>Misc Functions</h1>

In [2]:
cells = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def minutiae_at(pixels, i, j):
    values = [pixels[i + k][j + l] for k, l in cells]

    crossings = 0
    for k in range(0, 8):
        crossings += abs(values[k] - values[k + 1])
    crossings /= 2

    if pixels[i][j] == 0:
        if crossings == 1:
            return "ending"
        if crossings == 3:
            return "bifurcation"
    return "none"

def load_image(im):
    (x, y) = im.size
    im_load = im.load()

    result = []
    for i in range(0, x):
        result.append([])
        for j in range(0, y):
            result[i].append(im_load[i, j])

    return result

def apply_to_each_pixel(pixels, f):
    for i in range(0, len(pixels)):
        for j in range(0, len(pixels[i])):
            pixels[i][j] = f(pixels[i][j])
            
def detect_minutiaes(img_path):
    im = Image.open(img_path)
    pixels = load_image(im)
    apply_to_each_pixel(pixels, lambda x: 0.0 if x > 10 else 1.0)

    (x, y) = im.size
    result = im.convert("RGB")

    draw = ImageDraw.Draw(result)

    colors = {"ending" : (0, 255, 0), "bifurcation" : (255, 0, 0)}

    ellipse_size = 1
    
    for i in range(1, x - 1):
        for j in range(1, y - 1):
            minutiae = minutiae_at(pixels, i, j)
            if minutiae != "none":
                draw.ellipse([(i - ellipse_size, j - ellipse_size), (i + ellipse_size, j + ellipse_size)], outline = colors[minutiae])
    del draw

    return result

def saveModel(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

def loadModel():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
  
    return loaded_model

def performThinning(img_path):
    inp_image = imread(img_path)

    img_gray = rgb2gray(inp_image)

    img_gray=invert(img_gray)

    img_resized = resize(img_gray, (103, 96),anti_aliasing=True)

    thresh = threshold_otsu(img_resized)
    binary_thresh_img = img_resized > thresh

    #binary_thresh_img=binary_thresh_img.transpose(2,0,1).reshape(103,-1)

    skeleton = skeletonize(binary_thresh_img)
    
    return (skeleton,inp_image)
    

<h1>Image Preprocessing (Binarizing,Thinning)</h1>

In [185]:
for root, dirs, files in os.walk("fingerprint_dataset"):
    #char_list.append(os.path.basename(root).split('_')[-1])
    for f in files:
        print(f)
        skeleton = performThinning(os.path.join(root,f))[0]
        #out_thin = morphology.thin(binary_thresh_img)

        imsave("processed_fingerprint/"+f.split('.')[0]+".png", img_as_uint(skeleton))

100__M_Left_index_finger.BMP


ValueError: axes don't match array

<h1>Generating Class list to be Predicted</h1>

In [3]:
ext_list=[]
for root, dirs, files in os.walk("processed_fingerprint"):
    #char_list.append(os.path.basename(root).split('_')[-1])
    for f in files:
        part=f.split("__")
        if part[1].split(".")[0] not in ext_list:
            ext_list.append(part[1].split(".")[0])
print(ext_list)

['M_Left_index_finger', 'M_Left_little_finger', 'M_Left_middle_finger', 'M_Left_ring_finger', 'M_Left_thumb_finger', 'M_Right_index_finger', 'M_Right_little_finger', 'M_Right_middle_finger', 'M_Right_ring_finger', 'M_Right_thumb_finger', 'F_Left_index_finger', 'F_Left_little_finger', 'F_Left_middle_finger', 'F_Left_ring_finger', 'F_Left_thumb_finger', 'F_Right_index_finger', 'F_Right_little_finger', 'F_Right_middle_finger', 'F_Right_ring_finger', 'F_Right_thumb_finger']


<h1>Extracting Minutiae Points and Generating Dataset </h1>

In [ ]:
df = pd.DataFrame(columns=['pixel_'+str(i) for i in range(0,96*103)])
df['category']=''

In [ ]:
index=0
for root, dirs, files in os.walk("processed_fingerprint"):
    #char_list.append(os.path.basename(root).split('_')[-1])
    for f in files:
        print(f)
        im = Image.open(os.path.join(root,f))
        pixels = load_image(im)
        apply_to_each_pixel(pixels, lambda x: 0.0 if x > 10 else 1.0)

        (x, y) = im.size
        
        points = np.zeros((x,y),dtype=int)
        for i in range(1, x - 1):
            for j in range(1, y - 1):
                minutiae = minutiae_at(pixels, i, j)
                if minutiae != "none":
                    if minutiae == "ending":
                        points[i,j] = 1
                    if minutiae == "bifurcation":
                        points[i,j] = 2
                        
        pixel=list(points.flatten())
        pixel.append(f)
        df.loc[index]=pixel
        index += 1

In [ ]:
df.to_csv("data2.csv",index=False)

In [21]:
df=pd.read_csv("data2.csv")

<h1>Normalizing Category Column</h1>

In [ ]:
for i,v in enumerate(df['category']):
    ext = v.split('__')[-1].split('.')[0]
    print(ext)
    if ext in ext_list:
        print(ext_list.index(ext))
        df['category'].iloc[i]=ext_list.index(ext)

In [63]:
num_classes=20

(6000, 9889)

<h1>Splitting into Train and Test Set in 4:1 ratio</h1>

In [146]:
X=df.iloc[:,:] 
y=X['category']
X.drop(["category"], inplace = True, axis = 1)
#y=np.array(df.iloc[:,-1])
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.20,random_state=42)

In [148]:
X_train = X_train.reshape(X_train.shape[0], 96, 103 , 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 96, 103 , 1).astype('float32')

y_train = np_utils.to_categorical(y_train).astype('int32')
y_test = np_utils.to_categorical(y_test)

<h1>Building 2D CNN using Keras</h1>

In [153]:
model = Sequential()

model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(96, 103, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(70, kernel_size=3, padding="same", activation = 'relu'))
model.add(Conv2D(500, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(1024, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.3))

model.add(Dense(20))
model.add(Activation("softmax"))

In [154]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [168]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 96, 103, 40)       1040      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 51, 40)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 48, 51, 70)        25270     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 51, 500)       315500    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 25, 500)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 23, 1024)      4609024   
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 11, 11, 1024)      0         
__________

<h1>Training Neural Network</h1>

In [161]:
model.fit(X_train, y_train,
              batch_size=50,
              epochs=10,
              validation_data=(X_test, y_test),
                          shuffle=True)

Train on 4800 samples, validate on 1200 samples
Epoch 1/10
4800/4800 [==============================] - 1463s 305ms/step - loss: 1.7436 - acc: 0.4290 - val_loss: 1.9348 - val_acc: 0.3958
Epoch 2/10
4800/4800 [==============================] - 1448s 302ms/step - loss: 1.5956 - acc: 0.4783 - val_loss: 1.9410 - val_acc: 0.3850
Epoch 3/10
4800/4800 [==============================] - 1458s 304ms/step - loss: 1.4771 - acc: 0.5065 - val_loss: 2.0205 - val_acc: 0.3817
Epoch 4/10
4800/4800 [==============================] - 1369s 285ms/step - loss: 1.3409 - acc: 0.5538 - val_loss: 2.1353 - val_acc: 0.3908
Epoch 5/10
4800/4800 [==============================] - 1107s 231ms/step - loss: 1.2150 - acc: 0.5863 - val_loss: 2.2730 - val_acc: 0.3667
Epoch 6/10
4800/4800 [==============================] - 1103s 230ms/step - loss: 1.0548 - acc: 0.6410 - val_loss: 2.4280 - val_acc: 0.3708
Epoch 7/10
4800/4800 [==============================] - 1020s 212ms/step - loss: 0.9303 - acc: 0.6838 - val_loss: 2.54

In [162]:
scores = model.evaluate(X_test, y_test, verbose = 10 )
print(scores)

[3.086960916519165, 0.3541666666666667]


In [164]:
saveModel(model)

Saved model to disk


In [4]:
loaded_model=loadModel()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [5]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

<h1>Extracting Feature and Testing Input Fingerprint</h1>

In [12]:
img_path = "13__F_Left_ring_finger.png"

skeleton,inp_image = performThinning(img_path)

imsave("test.png", img_as_uint(skeleton))

tdf = pd.DataFrame(columns=['pixel_'+str(i) for i in range(0,96*103)])

In [13]:
im = Image.open("test.png")
pixels = load_image(im)
apply_to_each_pixel(pixels, lambda x: 0.0 if x > 10 else 1.0)

(x, y) = im.size

points = np.zeros((x,y),dtype=int)
for i in range(1, x - 1):
    for j in range(1, y - 1):
        minutiae = minutiae_at(pixels, i, j)
        if minutiae != "none":
            if minutiae == "ending":
                points[i,j] = 1
            if minutiae == "bifurcation":
                points[i,j] = 2

pixel=list(points.flatten())
tdf.loc[0]=pixel

In [9]:
f, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(10, 3))

ax0.imshow(inp_image, cmap='gray')
ax0.set_title('Input')

ax1.imshow(skeleton, cmap='gray')
ax1.set_title('Skeletonize')

out_thin = morphology.thin(skeleton)
ax2.imshow(out_thin, cmap='gray')
ax2.set_title('Thin')

minu = detect_minutiaes("test.png")
ax3.imshow(minu, cmap='gray')
ax3.set_title('Minutiae Detect')

#plt.savefig('smaple.png')

plt.show()

NameError: name 'plt' is not defined

<h1>Predicting Class</h1>

In [15]:
test = np.array(tdf.iloc[:,:])

test= test.reshape(1,96, 103 , 1).astype('float32')

res = loaded_model.predict(test)
res = np.argmax(res,axis = 1)
res = pd.Series(ext_list[res[0]], name="Category")
submission = pd.concat([pd.Series(range(1 ,2) ,name = "ImageId"),   res],axis = 1)
#submission.to_csv("cnn_mnist_datagen.csv",index=False)
submission.head(1)

,ImageId,Category
0,1,F_Left_ring_finger
